In [2]:
import dlib
import cv2
import numpy as np
import random
import face_recognition

cap = cv2.VideoCapture('./video/Alec_Baldwin.mp4') 
fps = cap.get(cv2.CAP_PROP_FPS)
total__frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#ms = cap.get(cv2.CAP_PROP_POS_MSEC)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))    #1280    # 取得畫面尺寸
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))   #720
font = 2;    lt = 10
print(fps,w,h,total__frames)

start_frame = 280
end_frame = 560
end_frame_sobal = 840
end_frame_mg = 1120
end_frame_4in1 = 1400
end_frame_4in1_camp = 1680
end_frame_Dog = 1960
end_frame_grammea = 2240
end_frame_face = 2520

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (w, h))
total_frames = end_frame - start_frame
scale_step = 1.0 / (end_frame - start_frame)
rotation_step = 4 * 360.0 / (end_frame - start_frame)#旋轉速度

colormaps = [cv2.COLORMAP_AUTUMN, cv2.COLORMAP_BONE, cv2.COLORMAP_JET, cv2.COLORMAP_WINTER,
             cv2.COLORMAP_RAINBOW, cv2.COLORMAP_OCEAN, cv2.COLORMAP_SUMMER, cv2.COLORMAP_SPRING,
             cv2.COLORMAP_COOL, cv2.COLORMAP_HSV, cv2.COLORMAP_PINK, cv2.COLORMAP_HOT,
             cv2.COLORMAP_PARULA, cv2.COLORMAP_MAGMA, cv2.COLORMAP_INFERNO, cv2.COLORMAP_PLASMA]

def get_random_colormaps():#變換colormap順序
    colormaps = list(range(0, 16))
    random.shuffle(colormaps)
    return colormaps

colormap_counter = 0
colormaps = get_random_colormaps()

def adjust_gamma(image, gamma):
    invgamma =1/(1/gamma)
    table = np.array([((i / 255.0) ** invgamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def apply_mosaic(face, mosaic_size=15):#方格馬賽克效果 
    face_height, face_width, _ = face.shape
    mosaic_face = face.copy()

    for y in range(0, face_height, mosaic_size):
        for x in range(0, face_width, mosaic_size):
            mosaic_cell = face[y:y + mosaic_size, x:x + mosaic_size]
            cell_color = np.mean(mosaic_cell, axis=(0, 1)).astype(int)
            mosaic_face[y:y + mosaic_size, x:x + mosaic_size] = cell_color

    return mosaic_face

face_detection_interval = 2 #人臉偵測間格
frames = [np.zeros((h//4, w//4, 3), dtype=np.uint8)] * 16 #處理邊界
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(5) == 27: break
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_Original = frame
    if start_frame <= frame_number < end_frame:
        #scale = 1.0 - (frame_number - start_frame) * scale_step
        scale =  (frame_number - start_frame) * scale_step
        rotation = (frame_number - start_frame) * rotation_step
        h, w = frame.shape[:2]
        center = (w // 2, h // 2)
        # 計算縮放和旋轉矩陣
        transform_matrix = cv2.getRotationMatrix2D(center, rotation, scale)
        # 應用變換
        frame = cv2.warpAffine(frame, transform_matrix, (w, h))
        cv2.putText(frame, f"2. Rotation: 4 & Scale {scale*rotation:.0f} deges.", 
                (10, 30), font, .8, (0,0,255), 2, lt)
    elif end_frame <= frame_number < end_frame_sobal:
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Split the frame into three equal parts
        width_third = w // 3
        left = gray_frame[:, :width_third]
        middle = gray_frame[:, width_third:2 * width_third]
        right = gray_frame[:, 2 * width_third:]
        # Apply edge detection methods to each part
        left_x = cv2.Sobel(left, cv2.CV_64F, 1, 0, ksize=3)
        left_y= cv2.Sobel(left, cv2.CV_64F, 0, 1, ksize=3)
        left_xy = cv2.addWeighted(left_x, 0.5, left_y, 0.5, 0) 
        middle = cv2.GaussianBlur(middle, (5, 5), 0)
        middle = cv2.Canny(middle, 30, 150)
        right = cv2.Laplacian(right, cv2.CV_64F)        
        left = cv2.convertScaleAbs(left_xy)
        middle = cv2.convertScaleAbs(middle)
        right = cv2.convertScaleAbs(right)
        # Merge the processed parts back together
        frame = cv2.hconcat([left, middle, right])  
        img1 = np.zeros_like(frame_Original)
        img1[:,:,0] = frame
        img1[:,:,1] = frame
        img1[:,:,2] = frame
        frame = img1
        cv2.putText(frame, "3. Sobal ", (10, 30), font, .8, (0,255,247), 2, lt)
        cv2.putText(frame, "Canny ", (10+width_third, 30), font, .8, (0,255,247), 2, lt)
        cv2.putText(frame, "Laplacian ", (10+width_third*2, 30), font, .8, (0,255,247), 2, lt)
    elif end_frame_sobal <= frame_number < end_frame_mg:
        k=np.ones((5, 5), np.uint8)
        frame = cv2.morphologyEx(frame, cv2.MORPH_GRADIENT, k)
        cv2.putText(frame, "4. Morph_Gradient", (10, 30), font, .8, (0,255,247), 2, lt)
        
    elif end_frame_mg <= frame_number < end_frame_4in1:
        new_width, new_height = w // 2, h // 2
        # Resize the image to the new dimensions
        frame_resized = cv2.resize(frame, (new_width, new_height))
        # Create the flipped images
        frame2 = cv2.flip(frame_resized, 0)  # Flip vertically
        frame3 = cv2.flip(frame_resized, 1)  # Flip horizontally
        frame4 = cv2.flip(frame2, 1)         # Flip both vertically and horizontally
        # Place the images in the correct positions in img1
        img1 = np.zeros_like(frame_Original)
        img1[0:new_height, 0:new_width] = frame_resized
        img1[0:new_height, new_width:w] = frame3
        img1[new_height:h, 0:new_width] = frame2
        img1[new_height:h, new_width:w] = frame4
        cv2.putText(img1, "5. 4 in 1 + Flip", (10, 30), font, .8, (0,255,247), 2, lt)
        frame = img1
    
    elif end_frame_4in1 <= frame_number < end_frame_4in1_camp:
        small_frame = cv2.resize(frame, (w//4, h//4))
        colormap_counter += 1
        if colormap_counter % 10 == 0:
            colormaps = get_random_colormaps()
        random_colormaps = random.sample(colormaps, len(colormaps))
        # Apply colormaps and create 4x4 grid
        for i in range(4):
            for j in range(4):
                frames[i*4+j] = cv2.applyColorMap(small_frame, colormaps[i*4+j])
        # Concatenate frames horizontally
        row1 = np.hstack(frames[:4])
        row2 = np.hstack(frames[4:8])
        row3 = np.hstack(frames[8:12])
        row4 = np.hstack(frames[12:])
        # Concatenate rows vertically
        result = np.vstack([row1, row2, row3, row4])
        cv2.putText(result, "6. 4x4 in 1 + Colormap", (10, 30), font, .8, (0, 255, 247), 2, lt)
        frame = result
    
    elif end_frame_4in1_camp <= frame_number < end_frame_Dog:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        d=5
        img_D3 = cv2.GaussianBlur(gray_frame, (d, d), 0.3)
        img_D5 = cv2.GaussianBlur(gray_frame, (d, d), 0.5)   
        img_D5_3 = img_D5 - img_D3   # try img_G0 + img_G1
        img1 = np.zeros_like(frame_Original)
        img1[:,:,0] = img_D5_3
        img1[:,:,1] = img_D5_3
        img1[:,:,2] = img_D5_3        
        cv2.putText(img1, "7. Dog", (10, 30), font, .8, (0, 255, 247), 2, lt)
        frame = img1
    
    elif end_frame_Dog <= frame_number < end_frame_grammea :
        new_width, new_height = w // 2, h // 2 
        frame_resized = cv2.resize(frame, (new_width, new_height))     
        corrected_img1 = adjust_gamma(frame_resized, 0.50)
        corrected_img2 = adjust_gamma(frame_resized, 0.75)
        corrected_img3 = adjust_gamma(frame_resized, 1.25)
        corrected_img4 = adjust_gamma(frame_resized, 1.50)
        # Place the images in the correct positions in img1
        img1 = np.zeros_like(frame_Original)
        img1[0:new_height, 0:new_width] = corrected_img1
        img1[0:new_height, new_width:w] = corrected_img2
        img1[new_height:h, 0:new_width] = corrected_img3
        img1[new_height:h, new_width:w] = corrected_img4
        cv2.putText(img1, "8. Gamma 0.50 ", (10, 30), font, .8, (0,255,247), 2, lt)
        cv2.putText(img1, "Gamma 0.75 ", (10+new_width, 30), font, .8, (0,255,247), 2, lt)
        cv2.putText(img1, "Gamma 1.25 ", (10, new_height+30), font, .8, (0,255,247), 2, lt)
        cv2.putText(img1, "Gamma 1.50 ", (10+new_width, new_height+30), font, .8, (0,255,247), 2, lt)  
        frame = img1
    
    elif end_frame_grammea <= frame_number < end_frame_face :
        face_detection_scale = 0.3
        scaled_frame = cv2.resize(frame, (0, 0), fx=face_detection_scale, fy=face_detection_scale)
        detector = dlib.get_frontal_face_detector()
        face_rects, scores, idx = detector.run(scaled_frame, 1, -0)
        for i, d in enumerate(face_rects):
            x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()
            x1, y1, x2, y2 = int(x1 / face_detection_scale), int(y1 / face_detection_scale), int(x2 / face_detection_scale), int(y2 / face_detection_scale)
            face = frame[y1:y2, x1:x2]
            blurred_face = apply_mosaic(face) #方格馬賽克
            #blurred_face = cv2.GaussianBlur(face, (25, 25), 15)  # 進行高斯模糊
            frame[y1:y2, x1:x2] = blurred_face  # 將模糊的臉部放回原始圖像
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA)
            cv2.putText(frame, f'{scores[i]:.2f}({idx[i]:0.0f})', (x1, y1), font, 0.7, (255, 255, 255), 1, lt)
        cv2.putText(frame, f"9. Face detection + Mosaic : {len(face_rects)}", (10, 30), font, .8, (0,255,247), 2, lt)
    
    elif end_frame_face <= frame_number < total__frames :
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sift=cv2.xfeatures2d.SIFT_create()    # create object
        kp = sift.detect(gray_frame, None)
        frame = cv2.drawKeypoints(gray_frame, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # img1 = np.zeros_like(frame_Original)
        # img1[:,:,0] = gray_frame
        # img1[:,:,1] = gray_frame
        # img1[:,:,2] = gray_frame 
        cv2.putText(frame, f"10. Sift : {len(kp)} kps ", (10, 30), font, .8, (0,255,247), 2, lt)     
    
    else:
        cv2.putText(frame, "1. Original", (10, 30), font, .8, (0,255,247), 2, lt)
    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f}/{total__frames} frames, FPS : {fps:.0f}',(10, 60), font, .8, (0,255,247), 2, lt)
    out.write(frame)
    cv2.imshow('frame',frame)
    frame_number += 1

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

30.0 1280 720 2803


-1